# Calidad del aire en Iberoamérica (últimos 90 días)
Comparación de **PM2.5** en cinco capitales: Santiago 🇨🇱, Ciudad de México 🇲🇽, Bogotá 🇨🇴, Madrid 🇪🇸 y Brasília 🇧🇷.

**Fuente:** [OpenAQ](https://openaq.org) · Este notebook asume que ya ejecutaste `python src/fetch_openaq_pm25.py` para generar `data/pm25_90d.csv`.


In [ ]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('../data/pm25_90d.csv', parse_dates=['date'])
df['date'] = pd.to_datetime(df['date'])
df.head()

## Mapa interactivo (promedio 90 días)
El tamaño y el color representan el promedio de PM2.5 por ciudad en el período.

In [ ]:
last_mean = (
    df.groupby(['country','city'], as_index=False)
      .agg(pm25_mean=('pm25','mean'),
           lat=('lat','median'),
           lon=('lon','median'))
)
fig_map = px.scatter_geo(
    last_mean,
    lat='lat', lon='lon',
    size='pm25_mean', color='pm25_mean',
    hover_name='city',
    projection='natural earth',
    title='PM2.5 promedio (últimos 90 días) – 5 capitales iberoamericanas',
)
fig_map.update_traces(marker=dict(opacity=0.85))
fig_map.update_layout(legend_title_text='PM2.5 (µg/m³)')
fig_map.show()

## Evolución diaria por ciudad
Se agrega una línea guía con el valor de referencia diario aproximado recomendado por la OMS (15 µg/m³).

In [ ]:
fig_line = px.line(
    df,
    x='date', y='pm25', color='city',
    labels={'pm25':'PM2.5 (µg/m³)', 'date':'Fecha'},
    title='Evolución diaria de PM2.5 (últimos 90 días)'
)
fig_line.add_hline(y=15, line_dash='dash',
                   annotation_text='Referencia OMS 24h (~15 µg/m³)',
                   annotation_position='top left')
fig_line.show()